In [2]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://urldefense.com/v3/__https://www.apache.org/licenses/LICENSE-2.0*5Cn__;JQ!!KEc8uF_xo8-al5zF!TFxM5JIZ9RmphkPHNnVGObbMz3_4wX82ie0o_wIrJ31_owKC5da5eEro18u5ec7Nt-gk-kJOCkDyjaE5$ #
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Installation

1. Install the necessary python Packages.  Note these workbenches come with pre-installed libraries so we might have to investigate the libraries we would need to install on a service
2. Set the project, region, and bucket_uri

In [3]:
# Need to install Python 3.8> for all the Langchain Libs to work
from platform import python_version
print(python_version())

3.10.11


In [4]:
# Install the packages
!pip install --upgrade --quiet google-cloud-aiplatform \
                        google-cloud-storage \
                        'google-cloud-bigquery[pandas]' \
                        typo \
                        google-cloud-storage

!pip install python-Levenshtein --quiet
!pip install --quiet pandas_gbq
!pip install --quiet langchain



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-python-client 1.8.0 requires google-api-core<2dev,>=1.13.0, but you have google-api-core 2.11.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install flake8 pycodestyle_magic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 9.1 MB/s eta 0:00:00


In [6]:
PROJECT_ID = "whisper-test-378918"

# Set the project id
# ! gcloud config set project {PROJECT_ID}
REGION = "us-central1"  # @param {type: "string"}
BUCKET_URI = "gs://whisper-test-fintech-demo"  # @param {type:"string"}

## Entity Extractor

In [7]:
# Import necessary libraries
from google.cloud import bigquery
from typing import TYPE_CHECKING, Any, Iterable, List, Optional, Type, Tuple, Dict
import json
import numpy as np
import pandas as pd
import pprint as pp


# BigQueryFetcher class
# Class for taking queries and creating dataframes or cleaned dictionaries (No Null keys/values)
class BigQueryFetcher:
    # Initialize BigQueryFetcher with the Google Cloud project ID
    def __init__(self, project_id: str):
        self.client = bigquery.Client(project=project_id)

    # Run the provided SQL query using BigQuery
    # and return the results as a dataframe
    def query_to_df(self, query: str) -> pd.DataFrame:
        query_job = self.client.query(query)
        rows = query_job.result()
        df = query_job.to_dataframe()

        return df

    def remove_null_keys_values(self, df: pd.DataFrame,
                                key: List[str]) -> Dict:
        df = df.where(pd.notnull(df), None)  # Replace pandas NaN with None
        # Convert DataFrame to list of dictionaries
        records = df.to_dict(orient='records')

        cleaned_records = {}
        for record in records:
            # For each record, create a new dictionary
            # that only includes keys with non-null values
            cleaned_record = {k: v for k, v in record.items() if v is not None}
            lei = cleaned_record.pop(key, None)
            if lei is not None:
                cleaned_records[lei] = cleaned_record

        return cleaned_records

    # Run the provided query, then clean the resulting data
    def df_to_clean_dict(self, query: str, key: List[str]) -> Dict:
        df = self.query_to_df(query)
        cleaned_data = self.remove_null_keys_values(df, key)
        return cleaned_data

In [9]:
#Subject to change if we want the datasource to come from a GCS bucket or a client request

bigquery_fetcher = BigQueryFetcher(PROJECT_ID)

##SQL query to retrive the transactions with reference
# Update the table & where clause based on the transactions with typos dataset
query = '''
SELECT APPLREF, REMITTANCE_INFO FROM `whisper-test-378918.fintech_dummy_data.transaction_typos_demo` 
where remittance_info not in ('PAY INVOICE', 'FOR SERVICES','OAT - OWN ACCOUNT TRANSFER  SERVICE','WITHDRAWAL TO BANK ACCOUNT') and 
remittance_info not like '%/BEN None%' and remittance_info like '%//BEN%'

ORDER BY 2 desc;
'''


transaction_demo = bigquery_fetcher.query_to_df(query)

In [10]:
transaction_demo.head()

,APPLREF,REMITTANCE_INFO
0,20230601EQHUJAXMNB,/RFB/9998477510811 //EUR 8.30237381948875/GBP ...
1,20230601FYCVRMQXFQ,/RFB/9997320282130 //SGD 10.970051323U14433/GB...
2,20230601JYICKWTCZV,/RFB/9995885640522 //EUR 2.418595817079784/GBP...
3,20230601OOJEHZOCGR,/RFB/9993714374527 //EUR 5.00001966544501/GBP ...
4,20230601RHQUJTDMMY,/RFB/9992740692343 //GBP 10.5824 01151471869/U...


In [13]:
# Used to Manage GCP quota limits for vertex models
#https://urldefense.com/v3/__https://cloud.google.com/vertex-ai/docs/quotas*request_quotas*5Cn__;IyU!!KEc8uF_xo8-al5zF!TFxM5JIZ9RmphkPHNnVGObbMz3_4wX82ie0o_wIrJ31_owKC5da5eEro18u5ec7Nt-gk-kJOCnqPlnlN$ 
def rate_limit(max_per_minute: int):
    period = 60 / max_per_minute
    print("Waiting")
    while True:
        before = time.time()
        yield
        after = time.time()
        elapsed = after - before
        sleep_time = max(0, period - elapsed)
        if sleep_time > 0:
            print(".", end="")
            time.sleep(sleep_time)

In [11]:
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain
from tqdm import tqdm
from typing import List, Tuple, NamedTuple, Dict
import pandas as pd
import time
from dataclasses import dataclass
import threading
from queue import Queue

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


# Define your  data structure that
# the entity extraction service will return
@dataclass
class Extraction:
    document_id: str
    entity: Dict

    
# Define your desired data structure for the returned json.
class entity_name(BaseModel):
    entity_name: str = Field(
        description="""
        Entity name that is extracted from string.
        The name can be in many languages
        """
    )


class EntityExtraction:
    """A class to summarize documents with LLM."""
    def __init__(self, requests_per_minute: int):
        self._llm = VertexAI(
            model_name="text-bison@001",
            max_output_tokens=1024,
            temperature=0,
            top_p=0.8,
            top_k=40,
            verbose=True,
        )
        self.entity_extraction_preamble = '''
        As an AI assistant, your task involves meticulously reviewing text and
        discerning the important details. In this case, your focus is to
        identify the entity names present in the input text.
        Create a JSON object with the extracted company names.
        Follow the format provided in the instructions below.
        In instances where the text doesn't include any company
        or individual names, simply return 'none'. Note that
        some texts may contain the abbreviation 'BEN',
        which stands for 'beneficiary'.

        Format instructions: {format_instructions}

        Sample:
        Input: {input_example}
        Output:
        '''
        self.parser = PydanticOutputParser(pydantic_object=entity_name)
        self._prompt = PromptTemplate(
                            input_variables=["input_example"],
                            template=self.entity_extraction_preamble,
                            partial_variables={"format_instructions":
                                               self.parser.get_format_instructions()})
        self.llm_chain = LLMChain(prompt=self._prompt, llm=self._llm)
        self.requests_per_minute = requests_per_minute

    def _extract_entities_thread(self, dataframe: pd.DataFrame, document_id: str, document_string: str,  extractions: Queue, documents_not_extracted: Queue) -> List[Tuple[int, str]]:
        """Extract entities from dataframe using LLM
        and return a list of namedtuples.

        Parameters:
            dataframe (pd.DataFrame): The dataframe to extract from.
            document_id (str): The column id.
            document_string (str): The document string to analyze.

        Returns:
            List[Tuple[int, str]]:
            A list of namedtuples where each tuple contains a column id and an entity.
        """
        limiter = rate_limit(self.requests_per_minute)
        # Extraction = namedtuple('Extraction',
        # [document_id, document_string])

        for idx, row in tqdm(dataframe.iterrows(),
                             total=dataframe.shape[0],
                             desc="Extracting entities"):
            try:
                string_to_extract = row[document_string]
                column_id = row[document_id]
                # print(self.llm_chain.prompt.format(input_example=string_to_extract))
                entity = self.llm_chain.run(string_to_extract)
                # print(entity)
                next(limiter)
                parsed_entity = self.parser.parse(entity)
                extraction = Extraction(column_id, parsed_entity)
                # extractions.append(extraction)
                extractions.put(extraction)  # Use the Queue's put method to add to the extractions

            except Exception as e:
                print(f"Failed to extract entity for row {idx}: {e}")
                document_not_extracted = column_id
        return extractions, document_not_extracted
    
    
    def extract_entities(self, dataframe: pd.DataFrame, document_id: str, document_string: str):
        extractions = Queue()  # Use a Queue to store the extractions
        documents_not_extracted  = Queue()
        thread = threading.Thread(target=self._extract_entities_thread, args=(dataframe, document_id, document_string, extractions, documents_not_extracted))
        thread.start()
        return extractions, documents_not_extracted, thread  # Return the Queue and the Thread

In [14]:
entity_extraction = EntityExtraction(60)

# Assumes that the document_id provided is unique to each row
# For demo purposes, only extracts 100 records
extractions, documents_not_extracted, thread = entity_extraction.extract_entities(transaction_demo.head(100), 'APPLREF', 'REMITTANCE_INFO')

extractions_list = []
documents_not_extracted_list = [] # Inputs that failed on extraction step
    


Extracting entities:   1%|          | 1/100 [00:00<00:58,  1.69it/s]

Waiting


Extracting entities:   2%|▏         | 2/100 [00:01<01:21,  1.20it/s]

..

Extracting entities:   4%|▍         | 4/100 [00:03<01:30,  1.06it/s]

...

Extracting entities:  98%|█████████▊| 98/100 [01:37<00:02,  1.00s/it]

.

Extracting entities:  99%|█████████▉| 99/100 [01:38<00:01,  1.00s/it]

.

Extracting entities: 100%|██████████| 100/100 [01:39<00:00,  1.00it/s]


In [15]:
# And if you want to ensure that the thread has finished, you can do:
thread.join()

while not extractions.empty():
    extractions_list.append(extractions.get())


while not documents_not_extracted.empty():
    documents_not_extracted_list.append(documents_not_extracted.get())


In [16]:
extractions_list[0:5]

[Extraction(document_id='20230601EQHUJAXMNB', entity=entity_name(entity_name='FIT ACTIVE BODY LTD')),
 Extraction(document_id='20230601FYCVRMQXFQ', entity=entity_name(entity_name='THE FITNESS EXCHANGE AT THOMAS MORE SQUARE LIMITED')),
 Extraction(document_id='20230601JYICKWTCZV', entity=entity_name(entity_name='BEN SIMMONS OF CHESHIRE LTD')),
 Extraction(document_id='20230601OOJEHZOCGR', entity=entity_name(entity_name='SKYANGELS AIR AMBULANCE GLOBAL LTD')),
 Extraction(document_id='20230601RHQUJTDMMY', entity=entity_name(entity_name='Megan Vargas'))]

## Create the Embeddings

1. Create a class to generate the embeddings. The class will process the list of structs in chunks of 5 to comply with the API limitations.
2. Save the embeddings to a bucket to be used for the index with matching engine.


In [210]:
from google.cloud import storage
import pandas as pd
import json
from langchain.embeddings import VertexAIEmbeddings
from langchain.embeddings.base import Embeddings
from langchain.llms import VertexAI
from langchain.prompts import PromptTemplate
from pydantic import BaseModel
import vertexai
import logging
import time
import uuid
import os
from io import BytesIO
import logging

logging.basicConfig(level=logging.INFO)


class CustomVertexAIEmbeddings(VertexAIEmbeddings, BaseModel):
    requests_per_minute: int
    num_instances_per_batch: int
    mins_per_batch: int = 10 # 10 mins is chosen to get 5*600*10 embeddings (30000)

    # Overriding embed_documents method
        # Overriding embed_documents method
    def embed_documents(self, texts: List[str]) -> List[float]:
        """Embed documents using the VertexAI API.

        Args:
            texts: List of documents to embed.

        Returns:
            List of embeddings.
        """
        limiter = rate_limit(self.requests_per_minute)
        results = []
        docs = list(texts)

        while docs:
            # Working in batches because the API accepts maximum 5
            # documents per request to get embeddings
            head, docs = (
                docs[: self.num_instances_per_batch],
                docs[self.num_instances_per_batch :],
            )
            try:
                chunk = self.client.get_embeddings(head)
                results.extend(chunk)
                next(limiter)
            except Exception as e:
                logging.error(f"Failed to get embeddings: {e}")
                continue

        return [r.values for r in results]

                
    def embed_and_save_documents(self, texts: List[str], document_ids: List[str], bucket_uri: str, unique_folder_name: str) -> None:
        """Embed documents and save them to GCS.

        Args:
            texts: List of documents to embed.
            document_ids: List of document IDs.
            bucket_uri: URI of the bucket to save to.
            unique_folder_name: Unique name for the folder to save to.
        """
        limiter = rate_limit(self.requests_per_minute)
        batch_size = self.num_instances_per_batch * self.requests_per_minute * self.mins_per_batch

        for i in tqdm(range(0, len(texts), batch_size)):  
            try:
                embeddings = self.embed_documents(texts[i: i + batch_size])
                self.write_embeddings_to_bucket(document_ids[i: i + batch_size], embeddings, bucket_uri, unique_folder_name)

                df = pd.DataFrame({
                    'document_id': document_ids[i: i + batch_size]
                })
                pp.pprint(df)
                self.write_parquet_to_bucket(bucket_uri, df)

                next(limiter)
            except Exception as e:
                logging.error(f"Failed to get embeddings: {e}")
                continue


    def write_embeddings_to_bucket(self, document_ids: List[str], embeddings: List[float], bucket_uri: str, unique_folder_name: str) -> None:
        """Write embeddings to a GCS bucket.

        Args:
            document_ids: List of document IDs.
            embeddings: List of embeddings.
            bucket_uri: URI of the bucket to write to.
            unique_folder_name: Unique name for the folder to write to.
        """
        bucket = storage.Client().bucket(bucket_uri.split('/')[-1])
        embeddings_str = "".join([json.dumps({"id": str(id), "embedding": vectors}) + "\n" for id, vectors in zip(document_ids, embeddings)])
        bucket.blob(f"{unique_folder_name}/{uuid.uuid4().hex}.json").upload_from_string(embeddings_str)


    def write_parquet_to_bucket(self, bucket_uri: str, df: pd.DataFrame) -> None:
        """Write parquet data to a GCS bucket.

        Args:
            bucket_uri: URI of the bucket to write to.
            df: DataFrame to write.
        """
        folder_name = "embedded_document_ids"
        bucket = storage.Client().bucket(bucket_uri.split('/')[-1])
        blob = bucket.blob(f"{folder_name}/{uuid.uuid4().hex}.parquet")
    
        # We need to create a BytesIO object because blob.upload_from_file requires a file-like object
        with io.BytesIO() as f:
            df.to_parquet(f)
            f.seek(0)
            blob.upload_from_file(f)


    def get_existing_ids(self, bucket_uri: str) -> pd.DataFrame:
        """Check for existing document ids and vector ids file in GCS and return a dataframe if it exists.

        Args:
            bucket_uri: URI of the bucket to check.

        Returns:
            DataFrame of existing ids.
        """
        storage_client = storage.Client()
        bucket_name = bucket_uri.split('/')[-1]
        bucket = storage_client.bucket(bucket_name)
        blobs = bucket.list_blobs(prefix='embedded_document_ids/')

        df_list = []
        for blob in blobs:
            if blob.name.endswith('.parquet'):
                byte_stream = BytesIO()
                blob.download_to_file(byte_stream)
                byte_stream.seek(0)
                df = pd.read_parquet(byte_stream)
                df_list.append(df)

        if df_list:
            return pd.concat(df_list)
        else:
            return pd.DataFrame()


In [211]:
vertexai.init(project=PROJECT_ID, location=REGION)

# Text model instance integrated with langChain
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=1024,
    temperature=0,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

# Embeddings API integrated with langChain
EMBEDDING_QPM = 600
EMBEDDING_NUM_BATCH = 5
embedding = CustomVertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
    mins_per_batch = 1
)

## Create Indexes & Deploy Endpoints

The following steps need to happen to deploy a Matching Engine endpoint:

1. Create an index. This class implements the tree-AH algorithm. It reads the embeddings generated from the earlier step in a GCP bucket in the correct format, as described in the [documentation](https://urldefense.com/v3/__https://cloud.google.com/vertex-ai/docs/matching-engine/match-eng-setup/format-structure*data-file-formats).*5Cn__;IyU!!KEc8uF_xo8-al5zF!TFxM5JIZ9RmphkPHNnVGObbMz3_4wX82ie0o_wIrJ31_owKC5da5eEro18u5ec7Nt-gk-kJOCs-Kw0rv$ 2. Create a Matching Engine Index Endpoint on your VPC Network (Not required).
3. Deploy the endpoint by associating it with a deployed Matching Engine Index.

**Note**: Depending on your use case, your setup might look different for creating and deploying an index. The main differentiator is whether you want to deploy your index to a public endpoint or to your own Virtual Private Cloud (VPC). If you decide to use a VPC, you can enable Private Service Connect or VPC Peering.

In [225]:
# Utility functions to create Index and deploy the index to an Endpoint
from datetime import datetime
import time
import logging
import tempfile

from google.cloud import aiplatform_v1 as aipv1, storage
from google.protobuf import struct_pb2

logging.basicConfig(level = logging.INFO)
logger = logging.getLogger()

class MatchingEngineUtils:
    def __init__(self,
                 project_id: str,
                 region: str,
                 index_name: str):
        self.project_id = project_id
        self.region = region
        self.index_name = index_name
        self.index_endpoint_name = f"{self.index_name}_endpoint"
        self.PARENT = f"projects/{self.project_id}/locations/{self.region}"

        ENDPOINT = f"{self.region}-aiplatform.googleapis.com"
        # set index client
        self.index_client = aipv1.IndexServiceClient(
            client_options=dict(api_endpoint=ENDPOINT)
        )
        # set index endpoint client
        self.index_endpoint_client = aipv1.IndexEndpointServiceClient(
            client_options=dict(api_endpoint=ENDPOINT)
        )

    def get_index(self):
        # Check if index exists
        request = aipv1.ListIndexesRequest(parent=self.PARENT)
        page_result = self.index_client.list_indexes(request=request)
        indexes = [response.name for response in page_result
                   if response.display_name == self.index_name]

        if len(indexes) == 0:
            return None
        else:
            index_id = indexes[0]
            request = aipv1.GetIndexRequest(name=index_id)
            index = self.index_client.get_index(request=request)
            return index

    def get_index_endpoint(self):
        # Check if index endpoint exists
        request = aipv1.ListIndexEndpointsRequest(parent=self.PARENT)
        page_result = self.index_endpoint_client.list_index_endpoints(request=request)
        index_endpoints = [response.name for response in page_result
                           if response.display_name == self.index_endpoint_name]

        if len(index_endpoints) == 0:
            return None
        else:
            index_endpoint_id = index_endpoints[0]
            request = aipv1.GetIndexEndpointRequest(name=index_endpoint_id)
            index_endpoint = self.index_endpoint_client.get_index_endpoint(request=request)
            return index_endpoint

        
    def create_index(self,
                     embedding_gcs_uri: str,
                     dimensions: int,
                     description: str,
                     index_update_method: str = "streaming",
                     index_algorithm:str = "tree-ah"
                     ):
        # Get index
        index = self.get_index()
        # Create index if does not exists
        if index:
            logger.info(f"Index {self.index_name} already exists with id {index.name}")
        else:
            logger.info(f"Index {self.index_name} does not exists. Creating index ...")

            if index_update_method == "streaming":
                index_update_method = aipv1.Index.IndexUpdateMethod.STREAM_UPDATE
            else:
                index_update_method = aipv1.Index.IndexUpdateMethod.BATCH_UPDATE

            treeAhConfig = struct_pb2.Struct(
                fields={
                    "leafNodeEmbeddingCount": struct_pb2.Value(number_value=1000),
                    "fractionLeafNodesToSearch": struct_pb2.Value(number_value=.05),
                }
            )
            if index_algorithm == "tree-ah":
                algorithmConfig = struct_pb2.Struct(
                    fields={"treeAhConfig": struct_pb2.Value(struct_value=treeAhConfig)}
                )
            else:
                algorithmConfig = struct_pb2.Struct(
                    fields={"bruteForceConfig": struct_pb2.Value(struct_value=struct_pb2.Struct())}
                )
            config = struct_pb2.Struct(
                fields={
                    "dimensions": struct_pb2.Value(number_value=dimensions),
                    "approximateNeighborsCount": struct_pb2.Value(number_value=150),
                    "distanceMeasureType": struct_pb2.Value(string_value="DOT_PRODUCT_DISTANCE"),
                    "algorithmConfig": struct_pb2.Value(struct_value=algorithmConfig),
                    "shardSize": struct_pb2.Value(string_value="SHARD_SIZE_SMALL"),
                    "featureNormType": struct_pb2.Value(string_value="UNIT_L2_NORM")
                }
            )
            metadata = struct_pb2.Struct(
                fields={
                    "config": struct_pb2.Value(struct_value=config),
                    "contentsDeltaUri": struct_pb2.Value(string_value=embedding_gcs_uri),
                }
            )

            index_request = {
                "display_name": self.index_name,
                "description": description,
                "metadata": struct_pb2.Value(struct_value=metadata),
                "index_update_method": index_update_method,
            }

            r = self.index_client.create_index(parent=self.PARENT,
                                               index=index_request)
            logger.info(f'Creating index with long running operation {r._operation.name}')

            # Poll the operation until it's done successfullly.
            logging.info("Poll the operation to create index ...")
            while True:
                if r.done():
                    break
                time.sleep(60)
                print('.', end='')

            index = r.result()
            logger.info(f"Index {self.index_name} created with resource name as {index.name}")

        return index


    def deploy_index(self,
                     machine_type: str = "e2-standard-2",
                     min_replica_count: int = 2,
                     max_replica_count: int = 10,
                     network: str = None):
        try:
            # Get index if exists
            index = self.get_index()
            if not index:
                raise Exception(f"Index {self.index_name} does not exists. Please create index before deploying.")

            # Get index endpoint if exists
            index_endpoint = self.get_index_endpoint()
            # Create Index Endpoint if does not exists
            if index_endpoint:
                logger.info(f"Index endpoint {self.index_endpoint_name} already exists with resource " +
                            f"name as {index_endpoint.name} and endpoint domain name as " +
                            f"{index_endpoint.public_endpoint_domain_name}")
            else:
                logger.info(f"Index endpoint {self.index_endpoint_name} does not exists. Creating index endpoint...")
                index_endpoint_request = {"display_name": self.index_endpoint_name}

                if network:
                    index_endpoint_request["network"] = network
                else:
                    index_endpoint_request["public_endpoint_enabled"] = True

                r = self.index_endpoint_client.create_index_endpoint(
                    parent=self.PARENT,
                    index_endpoint=index_endpoint_request)
                logger.info(f'Deploying index to endpoint with long running operation {r._operation.name}')

                logger.info("Poll the operation to create index endpoint ...")
                while True:
                    if r.done():
                        break
                    time.sleep(60)
                    print('.', end='')

                index_endpoint = r.result()
                logger.info(f"Index endpoint {self.index_endpoint_name} created with resource " +
                            f"name as {index_endpoint.name} and endpoint domain name as " +
                            f"{index_endpoint.public_endpoint_domain_name}")
        except Exception as e:
            logger.error(f"Failed to create index endpoint {self.index_endpoint_name}")
            raise e

        # Deploy Index to endpoint
        try:
            # Check if index is already deployed to the endpoint
            for d_index in index_endpoint.deployed_indexes:
                if d_index.index == index.name:
                    logger.info(f"Skipping deploying Index. Index {self.index_name}" +
                                f"already deployed with id {index.name} to the index endpoint {self.index_endpoint_name}")
                    return index_endpoint

            timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
            deployed_index_id = f"{self.index_name.replace('-', '_')}_{timestamp}"
            deploy_index = {
                "id": deployed_index_id,
                "display_name": deployed_index_id,
                "index": index.name,
                "dedicated_resources": {
                    "machine_spec": {
                        "machine_type": machine_type,
                        },
                    "min_replica_count": min_replica_count,
                    "max_replica_count": max_replica_count
                    }
            }
            logger.info(f"Deploying index with request = {deploy_index}")
            r = self.index_endpoint_client.deploy_index(
                index_endpoint=index_endpoint.name,
                deployed_index=deploy_index
            )

            # Poll the operation until it's done successfullly.
            logger.info("Poll the operation to deploy index ...")
            while True:
                if r.done():
                    break
                time.sleep(60)
                print('.', end='')

            logger.info(f"Deployed index {self.index_name} to endpoint {self.index_endpoint_name}")

        except Exception as e:
            logger.error(f"Failed to deploy index {self.index_name} to the index endpoint {self.index_endpoint_name}")
            raise e

        return index_endpoint

    def get_index_and_endpoint(self):
        # Get index id if exists
        index = self.get_index()
        index_id = index.name if index else ''

        # Get index endpoint id if exists
        index_endpoint = self.get_index_endpoint()
        index_endpoint_id = index_endpoint.name if index_endpoint else ''
        

        return index_id, index_endpoint_id

    def delete_index(self):
        # Check if index exists
        index = self.get_index()

        # create index if does not exists
        if index:
            # Delete index
            index_id = index.name
            logger.info(f"Deleting Index {self.index_name} with id {index_id}")
            self.index_client.delete_index(name=index_id)
        else:
            raise Exception("Index {index_name} does not exists.")

    def delete_index_endpoint(self):
        # Check if index endpoint exists
        index_endpoint = self.get_index_endpoint()

        # Create Index Endpoint if does not exists
        if index_endpoint:
            logger.info(f"Index endpoint {self.index_endpoint_name}  exists with resource " +
                        f"name as {index_endpoint.name} and endpoint domain name as " +
                        f"{index_endpoint.public_endpoint_domain_name}")

            index_endpoint_id = index_endpoint.name
            index_endpoint = self.index_endpoint_client.get_index_endpoint(name=index_endpoint_id)
            # Undeploy existing indexes
            for d_index in index_endpoint.deployed_indexes:
                logger.info(f"Undeploying index with id {d_index.id} from Index endpoint {self.index_endpoint_name}")
                request = aipv1.UndeployIndexRequest(
                    index_endpoint=index_endpoint_id,
                    deployed_index_id=d_index.id)
                r = self.index_endpoint_client.undeploy_index(request=request)
                response = r.result()
                logger.info(response)

            # Delete index endpoint
            logger.info(f"Deleting Index endpoint {self.index_endpoint_name} with id {index_endpoint_id}")
            self.index_endpoint_client.delete_index_endpoint(name=index_endpoint_id)
        else:
            raise Exception(f"Index endpoint {self.index_endpoint_name} does not exists.")

In [178]:
# Query your reference data and put it in a dataframe
# Update Query
# Limit to 100,000 records for demo purposes
reference_query =  'SELECT legal_entity_identifier, company_name_latin_alphabet FROM `whisper-test-378918.fintech_dummy_data.bvd_embedding_demo` LIMIT 100000'

reference_data = bigquery_fetcher.query_to_df(reference_query)
reference_data.head()



,legal_entity_identifier,company_name_latin_alphabet
0,CS006111,PERTH & KINROSS DISABILITY SPORT - SCIO
1,CE024877,THE WASHING MACHINE PROJECT FOUNDATION
2,CE023261,THE SPARKLE FOUNDATION UK
3,CS000485,BORDER GROUP OF RIDING FOR THE DISABLED (SCIO)
4,CS001757,ABOYNE & DISTRICT MEN'S SHED


In [215]:
# Create embeddings and write them to GCS
# Update column name if appropriate
# This will take a while # of records / (batch size * rate limit) 
# 100,000/(5*600) = 33.33 minutes

EMBEDDINGS_FOLDER_NAME = 'embeddings_batch_test'  # Update this if needed

# Fetch existing document ids from the GCS bucket
existing_ids_df = embedding.get_existing_ids(BUCKET_URI)

# Check if there are any existing ids, if yes, filter out those ids from the data to be processed
if not existing_ids_df.empty:
    filtered_reference_data = reference_data.loc[~reference_data.legal_entity_identifier.isin(existing_ids_df.document_id)]
    # Updating document_list and document_ids post filtering
    document_list = filtered_reference_data.company_name_latin_alphabet.to_list()
    document_ids = filtered_reference_data.legal_entity_identifier.to_list()
else:
    document_list = reference_data.company_name_latin_alphabet.to_list()
    embeddings = embedding.embed_documents(document_list)

# This initiates the embedding and saving process
embedding.embed_and_save_documents(document_list, document_ids, BUCKET_URI, EMBEDDINGS_FOLDER_NAME)

0it [00:00, ?it/s]


In [217]:
embeddings_gcs_uri = f"{BUCKET_URI}/{EMBEDDINGS_FOLDER_NAME}/"

# Build Matching Engine Index

In [218]:
# This will take a while
ME_INDEX_NAME = 'fintech_entity_resolution_demo'
mengine = MatchingEngineUtils(PROJECT_ID, REGION, ME_INDEX_NAME)

reference_data_index = mengine.create_index(embeddings_gcs_uri,
                     dimensions=768, # number of embedding features len(embeddings[0]), 
                     description="Fintech Demo",
                     index_update_method="batch",
                     index_algorithm="tree-ah"
                    )
    
# ME_INDEX_ID, ME_INDEX_ENDPOINT_ID = mengine.get_index_and_endpoint()



INFO:root:Index fintech_entity_resolution_demo_l2 does not exists. Creating index ...
INFO:root:Creating index with long running operation projects/209480788681/locations/us-central1/indexes/8846820090667073536/operations/1487414038655664128
INFO:root:Poll the operation to create index ...


......................................................

INFO:root:Index fintech_entity_resolution_demo_l2 created with resource name as projects/209480788681/locations/us-central1/indexes/8846820090667073536


.

In [224]:
ME_INDEX_NAME = 'fintech_demo'
mengine = MatchingEngineUtils(PROJECT_ID, REGION, ME_INDEX_NAME)
mengine.get_index()

name: "projects/209480788681/locations/us-central1/indexes/7670149134900264960"
display_name: "fintech_demo"
metadata_schema_uri: "gs://google-cloud-aiplatform/schema/matchingengine/metadata/nearest_neighbor_search_1.0.0.yaml"
metadata {
  struct_value {
    fields {
      key: "config"
      value {
        struct_value {
          fields {
            key: "algorithmConfig"
            value {
              struct_value {
                fields {
                  key: "treeAhConfig"
                  value {
                    struct_value {
                      fields {
                        key: "leafNodeEmbeddingCount"
                        value {
                          string_value: "500"
                        }
                      }
                      fields {
                        key: "leafNodesToSearchPercent"
                        value {
                          number_value: 80.0
                        }
                      }
                    }

In [86]:
import requests

metadata_server_url = "https://urldefense.com/v3/__http://metadata.google.internal/computeMetadata/v1/project/numeric-project-id*5C*22*5Cn*22__;JSUlJQ!!KEc8uF_xo8-al5zF!TFxM5JIZ9RmphkPHNnVGObbMz3_4wX82ie0o_wIrJ31_owKC5da5eEro18u5ec7Nt-gk-kJOCoYzQzKU$ metadata_headers = {"Metadata-Flavor": "Google"}

response = requests.get(metadata_server_url, headers=metadata_headers)
project_number = response.content.decode()

VPC_NAME = 'text-matching-engine-test'
full_network = f"projects/{project_number}/global/networks/{VPC_NAME}"



In [87]:
# Deploy the index
index_endpoint = mengine.deploy_index(
                     machine_type = "e2-standard-16",
                     min_replica_count = 2,
                     max_replica_count = 10,
                     network = full_network
                    )

INFO:root:Index endpoint fintech_entity_resolution_demo_endpoint does not exists. Creating index endpoint...
INFO:root:Deploying index to endpoint with long running operation projects/209480788681/locations/us-central1/indexEndpoints/3086716167260209152/operations/2016569402685652992
INFO:root:Poll the operation to create index endpoint ...
INFO:root:Index endpoint fintech_entity_resolution_demo_endpoint created with resource name as projects/209480788681/locations/us-central1/indexEndpoints/3086716167260209152 and endpoint domain name as 
INFO:root:Deploying index with request = {'id': 'fintech_entity_resolution_demo_20230711010839', 'display_name': 'fintech_entity_resolution_demo_20230711010839', 'index': 'projects/209480788681/locations/us-central1/indexes/1991778507855757312', 'dedicated_resources': {'machine_spec': {'machine_type': 'e2-standard-16'}, 'min_replica_count': 2, 'max_replica_count': 10}}


.

INFO:root:Poll the operation to deploy index ...


....

INFO:root:Deployed index fintech_entity_resolution_demo to endpoint fintech_entity_resolution_demo_endpoint


.

In [32]:
# Class for retrieving ANN results from Matching Engine
# Requires an index endpoint to be deployed along with deployed index id
class MatchingEngineRetriever():
    def __init__(
            self,
            index_endpoint: str, 
            embedding: Embeddings 
            ):
        from google.cloud import aiplatform
        self.index_endpoint = aiplatform.MatchingEngineIndexEndpoint(index_endpoint_name=index_endpoint)
        self.deployed_index_id = self.index_endpoint.deployed_indexes[-1].id #returns the latest deploy index
        self.embedding = embedding
    
    
    def retrieve_results(self, num_neighbors: int, queries: list) -> List[List]:
        embedding_queries = self.embedding.embed_documents(queries)
        response = self.index_endpoint.match(
            deployed_index_id=self.deployed_index_id,
            queries=embedding_queries,
            num_neighbors=num_neighbors,
        )
    
        return response


In [33]:
me_retriver = MatchingEngineRetriever(
    index_endpoint=ME_INDEX_ENDPOINT_ID, 
    embedding =embedding )

## Entity Resolution

In [45]:
from tqdm.auto import tqdm
from dataclasses import dataclass
import uuid

@dataclass
class EntityResolution:
    uuid: str
    document_id: str
    entity_name: str
    nearest_neighbor_id: int
    nearest_neighbor: str
    distance_score: float
    prompt: str
    classification: str


class EntityResolutionSystem:
    def __init__(self, retriever: MatchingEngineRetriever, requests_per_minute: int, canonical_df: pd.DataFrame):
        self.me_retriver = retriever
        self._llm = VertexAI(
            model_name="text-bison@001",
            max_output_tokens=1024,
            temperature=0.2,
            top_p=0.8,
            top_k=40,
            verbose=True,
        )
        self.requests_per_minute = requests_per_minute
        self.canonical_df = canonical_df
        self._llm_chain = self._prepare_prompt()
        
    
    def _prepare_prompt(self) -> LLMChain:
        """
        Prepare the prompt to be used for entity matching
        """
        preamble = '''
        Given the two entities separated by the [SEP] token, determine whether they represent the same entity or different ones. 
        If they represent the same entity, the output should be 'match'. 
        If they represent different entities, the output should be 'no match'.
        Remember, your task is to evaluate the entities and classify them as either 'match' or 'no match' based on their similarity.
        Remember, this task demands an in-depth evaluation and a keen sense of judgement.
        Here are some examples:
        '''

        from langchain.prompts.few_shot import FewShotPromptTemplate
        from langchain.prompts.prompt import PromptTemplate


        examples = [
          {
            "input": "FIT ACTIVE BODY LTD [SEP] FITNESS INCORPORATED LTD ",
            "output": 'no match'
          }
        ]

        suffix = '''
            Given the following pair of entities:
            Input: {input} 
            output:'''

        example_formatter_template = """
        input: {input} 
        output: {output}"""

        example_prompt = PromptTemplate(input_variables=["input", "output"], template=example_formatter_template)


        prompt = FewShotPromptTemplate(
            examples=examples, 
            example_prompt=example_prompt,
            prefix=preamble,
            suffix=suffix, 
            input_variables=["input"]
        )
        
        llm_chain = LLMChain(prompt=prompt, llm=self._llm)

        return llm_chain
        
        
    def _filter_right_entities (self, rights_ids) -> List[str]:
        """
        Filter and return right entities from canonical_df based on provided ids
        """
        # Get a DataFrame of the rows where 'key' is in the list 'keys'
        matched_rows_df = self.canonical_df[self.canonical_df.iloc[:,0].isin(rights_ids)]

        # Get the column name
        column_name = matched_rows_df.columns[-1]

        # Extract the 'value' column from the matched rows as a list and prefix each value with column name
        # return [f'{column_name}={name}' for name in matched_rows_df.iloc[:,-1].tolist()]
        return matched_rows_df.iloc[:,-1].tolist()


    def _serialize_pair(entry1, entry2):
        CLS = "[CLS]entity_name="
        SEP = "[SEP]"

        serialized_entry1 = serialize(entry1)
        serialized_entry2 = serialize(entry2)

        return f"{CLS} {serialized_entry1} {SEP} {CLS} {serialized_entry2}"
    

    def match_entities(self, ids: list, queries: List, num_neighbors: int=5, distance_threshold: int=0) -> List[EntityResolution]:
        """
        Match entities and return a list of resolved entities
        """
        entities_resolved = []
        limiter = rate_limit(self.requests_per_minute)
        nearest_neighbors_results = self.me_retriver.retrieve_results(num_neighbors=num_neighbors, queries=queries)

        # create a tqdm progress bar with a total and a description
        progress_bar = tqdm(total=len(ids), desc="Matching entities", dynamic_ncols=True)

        #all 3 lists should have the same lenght
        for id, query, nearest_neighbors in zip(ids, queries, nearest_neighbors_results):
            left_entity = str(query.entity_name)
            left_id = id

            right_ids = [neighbor.id for neighbor in nearest_neighbors if neighbor.distance >= distance_threshold]
            right_entities = self._filter_right_entities(right_ids)
            nearest_neighbors_distance_scores = [neighbor.distance for neighbor in nearest_neighbors if neighbor.distance >= distance_threshold]
            
            #Loop through the top N neighbors for classiciation
            for right_id, right_entity, nearest_neighbor_distance_score in zip(right_ids,right_entities, nearest_neighbors_distance_scores):
            # prepare a unique identifier for each prompt
                entity_pair_to_match = left_entity + " [SEP] " + right_entity
                # entity_pair_to_match = _serialize_pair(left_entity, right_entity)

                #Update the dataclass
                entities_resolved.append(EntityResolution(
                    uuid = str(uuid.uuid4()),
                    document_id = left_id,
                    entity_name = left_entity,
                    nearest_neighbor_id = right_id,
                    nearest_neighbor = right_entity,
                    distance_score = nearest_neighbor_distance_score,
                    prompt = self._llm_chain.prompt.format(input=entity_pair_to_match),
                    classification = self._llm_chain.run(entity_pair_to_match)
                ))
                next(limiter)
                if len(entities_resolved) > 50:
                    self._write_results_to_bq(entities_resolved)
                    entities_resolved = []

            # update the progress bar
            progress_bar.update()

        # close the progress bar
        progress_bar.close()

        return entities_resolved

    def _write_results_to_bq(self, entities_resolved: List):
    # Define the table name in the format 'project_id.dataset_id.table_id'
        from dataclasses import asdict
        import pandas_gbq
        entities_resolved_dict = [asdict(resolved_entity) for resolved_entity in entities_resolved]
        df = pd.DataFrame(entities_resolved_dict)

        project_id = 'whisper-test-378918'
        dataset_id = 'fintech_dummy_data'
        table_id = 'entity_reconciliation_demo_new'

        table_name = f'{project_id}.{dataset_id}.{table_id}'
        pandas_gbq.to_gbq(df, table_id, project_id=project_id, if_exists='append')
    
    

    

In [ ]:
entity_matcher = EntityResolutionSystem(retriever=me_retriver, requests_per_minute=55, canonical_df=canonical_data)

resolved_entities = entity_matcher.match_entities(
    ids = [entity_extraction.document_id for entity_extraction in extractions_list],
    queries = [entity_extraction.entity for entity_extraction in extractions_list]
    )




Waiting
.............



Matching entities:   0%|          | 0/99 [00:00<?, ?it/s]

Waiting
....



Matching entities:   1%|          | 1/99 [00:05<08:32,  5.23s/it]

....

...

.....



Matching entities:   5%|▌         | 5/99 [00:27<08:40,  5.53s/it]

....



Matching entities:   6%|▌         | 6/99 [00:34<09:11,  5.93s/it]

....

In [593]:

import pandas_gbq
def write_results_to_bq(df, project_id, dataset_id, table_id):
    # Define the table name in the format 'project_id.dataset_id.table_id'
    table_name = f'{project_id}.{dataset_id}.{table_id}'
    pandas_gbq.to_gbq(df, table_name, project_id=project_id, if_exists='replace')
    
    
write_results_to_bq(df, project_id, dataset_id, table_id)






25 out of 25 rows loaded.00<?, ?it/s]INFO:pandas_gbq.gbq:
100%|██████████| 1/1 [00:00<00:00, 917.59it/s]


In [729]:
len(resolved_entities)

25

In [581]:
pp.pprint('l')

'l'
